In [1]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import chi2
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

#  Saving Features as pandas dataframe

In [2]:
df = pd.read_csv('extracted_feature_values_midi.csv')

In [3]:
cluster_mood_filename = df.cluster_mood_filename
temp = []
cluster = []
mood = []
filename = []

for i in cluster_mood_filename:
    temp.append(i)
temp = [i.split('\\') for i in temp]

for path in temp:
    cluster.append(path[-3])
    mood.append(path[-2])
    filename.append(path[-1].split('.')[0])

df['cluster'] = cluster
df['mood'] = mood
df['filename'] = filename

cols = [col for col in df.columns]
xcols = cols[1:-3]
ycol = cols[-3]
print(ycol)

cluster


#  Applying label encoder and splitting the dataset into train and test datasets

In [4]:
le = preprocessing.LabelEncoder()
X = df[xcols]
X = X.fillna(0)
Y = df[ycol]
Y = pd.DataFrame(Y)
Y = Y.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')
files = df['filename']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
# print(type(df))
# print(type(X_train))
# print(type(Y_train))
# print(type(xcols))
# print(type(ycol))
# print(X.isnull().sum())
# print(Y.isnull().sum())
# print(X.shape)
# X_nonan = X.fillna(0)
# print(X_nonan.isnull().values.any())
# print(X_nonan.shape)
print(X_train.shape)
print(Y_train.shape)
X_train.head()


(156, 1495)
(156, 1)


,Basic_Pitch_Histogram_0,Basic_Pitch_Histogram_1,Basic_Pitch_Histogram_2,Basic_Pitch_Histogram_3,Basic_Pitch_Histogram_4,Basic_Pitch_Histogram_5,Basic_Pitch_Histogram_6,Basic_Pitch_Histogram_7,Basic_Pitch_Histogram_8,Basic_Pitch_Histogram_9,...,Parallel_Motion,Similar_Motion,Contrary_Motion,Oblique_Motion,Parallel_Fifths,Parallel_Octaves,Dynamic_Range,Variation_of_Dynamics,Variation_of_Dynamics_In_Each_Voice,Average_Note_to_Note_Change_in_Dynamics
115,0.0,0,0,0,0,0,0,0,0,0,...,0.11490,0.25650,0.02904,0.5995,0.019360,0.03690,120,28.98,5.482,3.148
179,0.0,0,0,0,0,0,0,0,0,0,...,0.76400,0.07155,0.05839,0.1061,0.104400,0.24010,83,17.44,2.638,1.444
10,0.0,0,0,0,0,0,0,0,0,0,...,0.18430,0.09085,0.07778,0.6471,0.024840,0.04052,0,0.00,0.000,0.000
74,0.0,0,0,0,0,0,0,0,0,0,...,0.07518,0.34640,0.18620,0.3922,0.006347,0.01666,116,18.77,6.408,3.288
13,0.0,0,0,0,0,0,0,0,0,0,...,0.08871,0.23650,0.03538,0.6394,0.005187,0.01942,103,28.13,6.447,17.460


#  Normalizing the dataset for improved svm performance

In [5]:
#Normalizing using minmaxscalor scalor 
normalized_X = X.values
std_scalor = preprocessing.MinMaxScaler()
X_scaled = std_scalor.fit_transform(normalized_X)
normalized_X = pd.DataFrame(X_scaled, columns=xcols)
#X_train3, X_test3, Y_train3, Y_test3 = train_test_split(normalized_X, Y, test_size=0.3)
X_train3, X_test3, Y_train3, Y_test3 = train_test_split(normalized_X, Y, test_size=0.3, stratify = Y)
normalized_X.head()

,Basic_Pitch_Histogram_0,Basic_Pitch_Histogram_1,Basic_Pitch_Histogram_2,Basic_Pitch_Histogram_3,Basic_Pitch_Histogram_4,Basic_Pitch_Histogram_5,Basic_Pitch_Histogram_6,Basic_Pitch_Histogram_7,Basic_Pitch_Histogram_8,Basic_Pitch_Histogram_9,...,Parallel_Motion,Similar_Motion,Contrary_Motion,Oblique_Motion,Parallel_Fifths,Parallel_Octaves,Dynamic_Range,Variation_of_Dynamics,Variation_of_Dynamics_In_Each_Voice,Average_Note_to_Note_Change_in_Dynamics
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.24310,0.447814,0.209200,0.483782,0.094401,0.307831,0.550802,0.464353,0.273599,0.215714
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.05970,0.309545,0.000000,0.878854,0.088922,0.044794,0.336898,0.778531,0.034983,0.003240
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.356824,1.000000,0.458453,0.000000,0.000000,1.000000,0.832475,0.997878,0.479408
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.04642,0.331311,0.189775,0.793123,0.003734,0.016940,0.395722,0.593115,0.266002,0.217353
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.07807,0.349331,0.129150,0.773066,0.010256,0.137774,0.534759,0.111900,0.059720,0.002270


# SVM classification on original dataset without feature selection

In [6]:
# %%time
# #clf = LinearSVC(max_iter = 10000000, verbose = 1)
# clf = svm.SVC(C=1, kernel='linear', verbose = True)
# clf.fit(X_train, Y_train.values.ravel())
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=20)
# #cv = StratifiedKFold(5)
# #cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=20, random_state=1)
# train_accuracy_svm = cross_val_score(clf, X_train, Y_train.values.ravel(), cv=cv, n_jobs=15).mean()
# test_accuracy_svm = clf.score(X_test, Y_test.values.ravel())

# print("Training accuracy " + str(train_accuracy_svm))
# print("Test accuracy " + str(test_accuracy_svm))

# SVM classification on normalized dataset without feature selection

In [7]:
%%time
# clf = svm.SVC(C=0.1, kernel='linear', verbose = True)
# #clf = svm.SVC(C=0.1, kernel='rbf', verbose = True)
# #clf = LinearSVC(C=0.1, max_iter = 10000000)
# clf.fit(X_train3, Y_train3.values.ravel())
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=20)
# #cv = StratifiedKFold(5)
# #cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=20, random_state=1)
# train_accuracy_svm_norm = cross_val_score(clf, X_train3, Y_train3.values.ravel(), cv=cv, n_jobs=15).mean()
# test_accuracy_svm_norm = clf.score(X_test3, Y_test3.values.ravel())

param = {'C': [0.001,0.01,0.1, 1, 10, 100, 1000],
              'kernel': ['linear']}
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=20)
grid = GridSearchCV(svm.SVC(), param, verbose = 3, cv=cv, return_train_score=True, n_jobs=15)
grid.fit(X_train3, Y_train3.values.ravel())
    
train_accuracy_svm_norm = grid.best_score_
    
grid_predict3 = grid.predict(X_test3)
test_accuracy_svm_norm  = metrics.accuracy_score(Y_test3,grid_predict3)

print("Training accuracy " + str(train_accuracy_svm_norm))
print("Test accuracy " + str(test_accuracy_svm_norm))
print(grid.best_params_)

Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Training accuracy 0.33027472527472523
Test accuracy 0.3220338983050847
{'C': 0.01, 'kernel': 'linear'}
Wall time: 22.2 s


#  Feature Selection on Midi features

Methods to employ -<br> 1) Tree based feature selection(Random Forests) <br>
                    2) Recursive feature elimination with cross validation using SVM estimators <br>
                    3) Anova F-measures <br>
                    4) Chi-squared <br>
                    5) Principal component analysis<br>
We will apply each of them individually, evaluate performance, and choose a combination
Principal component analysis should be performed as the dimensions of the feature vectore(1495) is more than the number of samples(196) <br>

**study showed svm classification based on midi features had a test accuracy of 35%**

https://www.kaggle.com/arrohit/feature-selection-and-hypertuning-svm  **tree based feature selection**

In [8]:
%%time
rf_clf = RandomForestClassifier()
rf_clf = rf_clf.fit(X_train3, Y_train3.values.ravel())
model = SelectFromModel(rf_clf, prefit=True)

rf_X_train = X_train3.loc[:, model.get_support()]
rf_X_test = X_test3.loc[:, model.get_support()]
print(type(rf_X_train))
print(rf_X_train.shape)
rf_X_train.head()

<class 'pandas.core.frame.DataFrame'>
(137, 522)
Wall time: 160 ms


,Basic_Pitch_Histogram_28,Basic_Pitch_Histogram_29,Basic_Pitch_Histogram_33,Basic_Pitch_Histogram_35,Basic_Pitch_Histogram_37,Basic_Pitch_Histogram_38,Basic_Pitch_Histogram_41,Basic_Pitch_Histogram_42,Basic_Pitch_Histogram_44,Basic_Pitch_Histogram_45,...,Voice_Overlap,Variability_of_Voice_Separation,Parallel_Motion,Similar_Motion,Contrary_Motion,Parallel_Fifths,Parallel_Octaves,Variation_of_Dynamics,Variation_of_Dynamics_In_Each_Voice,Average_Note_to_Note_Change_in_Dynamics
174,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.378239,0.000000,0.000000,0.000000,...,0.512724,0.451650,0.652000,0.155772,0.139325,0.527338,0.239664,0.534020,0.268124,0.501799
72,0.000000,0.712098,0.000000,0.000000,0.000000,0.026009,0.603169,0.000000,0.021104,0.160385,...,0.496230,0.148140,0.004149,0.636574,0.238600,0.000000,0.012448,0.456492,0.376910,0.429828
45,0.000000,0.012660,0.000000,0.033444,0.000000,0.001494,0.050754,0.000000,0.000000,0.008677,...,0.874489,0.503143,0.143400,0.293310,0.152975,0.000000,0.413141,0.553538,0.542445,0.467413
153,0.050759,0.000000,0.010314,0.369960,0.183002,0.000000,0.000000,0.538703,0.139060,0.005907,...,0.895696,0.401676,0.252100,0.505263,0.188900,0.000000,0.668767,0.354297,0.413582,0.431427
178,0.045103,0.000000,0.148991,0.004746,0.000000,0.063299,0.118451,0.000000,0.000683,0.087352,...,0.956331,0.515715,0.047250,0.320785,0.121025,0.020596,0.038044,0.515316,0.459677,0.355458


In [9]:
%%time
# clf = svm.SVC(C=0.1, kernel='linear', verbose = True)
# #clf = svm.SVC(C=0.1, kernel='rbf', verbose = True)
# #clf = LinearSVC(C=0.1, max_iter = 10000000)
# clf.fit(rf_X_train, Y_train3.values.ravel())
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=20)
# #cv = StratifiedKFold(5)
# #cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=20, random_state=1)
# train_accuracy_svm_rf = cross_val_score(clf, rf_X_train, Y_train3.values.ravel(), cv=cv, n_jobs=15).mean()
# test_accuracy_svm_rf = clf.score(rf_X_test, Y_test3.values.ravel())

param = {'C': [0.001,0.01,0.1, 1, 10, 100, 1000],
              'kernel': ['linear']}
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=20)
grid = GridSearchCV(svm.SVC(), param, verbose = 3, cv=cv, return_train_score=True, n_jobs=15)
grid.fit(rf_X_train, Y_train3.values.ravel())
    
train_accuracy_svm_rf = grid.best_score_
    
grid_predict3 = grid.predict(rf_X_test)
test_accuracy_svm_rf  = metrics.accuracy_score(Y_test3,grid_predict3)

print("Training accuracy " + str(train_accuracy_svm_rf))
print("Test accuracy " + str(test_accuracy_svm_rf))
print(grid.best_params_)

Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Training accuracy 0.37252747252747254
Test accuracy 0.4406779661016949
{'C': 1, 'kernel': 'linear'}
Wall time: 2.81 s


https://machinelearningmastery.com/rfe-feature-selection-in-python/ **RFECV**

In [10]:
%%time
clf = svm.SVC(C=0.1, kernel='linear')
#clf = LinearSVC(C=0.1, max_iter = 10000000)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=20)
#cv = StratifiedKFold(5)
rfecv = RFECV(estimator = clf, step = 1, cv = cv, scoring='accuracy', n_jobs=15, min_features_to_select = 10 )
rfetrain=rfecv.fit(X_train3, Y_train3.values.ravel())
print('Optimal number of features :', rfecv.n_features_)

Optimal number of features : 38
Wall time: 6min 46s


In [11]:
%%time
#clf = svm.SVC(C=0.1, kernel='rbf', verbose = True)
rfe = RFE(estimator=clf, n_features_to_select=rfecv.n_features_, step=1)
rfe = rfe.fit(X_train3, Y_train3.values.ravel())

rfe_X_train = X_train3.loc[:, rfe.get_support()]
rfe_X_test = X_test3.loc[:, rfe.get_support()]

Wall time: 13.1 s


In [12]:
rfe_X_train.head()

,Pitch_Class_Histogram_8,Average_Range_of_Glissandos,Number_of_Common_Melodic_Intervals,Melodic_Thirds,Vertical_Interval_Histogram_24,Vertical_Minor_Third_Prevalence,Simple_Initial_Meter,Compound_Initial_Meter,Prevalence_of_Short_Rhythmic_Values,Median_Rhythmic_Value_Offset,...,Pitched_Instruments_Present_66,Unpitched_Instruments_Present_2,Unpitched_Instruments_Present_4,Unpitched_Instruments_Present_9,Unpitched_Instruments_Present_10,Unpitched_Instruments_Present_22,Unpitched_Instruments_Present_24,Unpitched_Instruments_Present_28,Unpitched_Instruments_Present_30,Time_Prevalence_of_Pitched_Instruments_30
174,0.000000,0.000000,0.6,0.499172,0.484482,0.695657,1.0,0.0,0.943557,0.066664,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8571
72,0.000000,0.000000,0.6,0.729359,0.444221,0.530384,1.0,0.0,0.768311,0.333360,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0000
45,0.000000,0.000000,0.8,0.384197,0.401302,0.297692,1.0,0.0,0.867353,0.333360,...,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0000
153,0.024911,0.000000,0.8,0.790395,0.302442,0.174342,1.0,0.0,0.677790,0.666640,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0000
178,0.211717,0.947031,1.0,0.541519,0.361584,0.555679,1.0,0.0,0.897527,0.333360,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0000


In [13]:
for index, rank in enumerate(rfe.ranking_):
    if rank in [1,15]:
        print("Rank " + str(rank)+ " feature: " + xcols[index])

Rank 1 feature: Pitch_Class_Histogram_8
Rank 1 feature: Average_Range_of_Glissandos
Rank 1 feature: Number_of_Common_Melodic_Intervals
Rank 1 feature: Melodic_Thirds
Rank 1 feature: Vertical_Interval_Histogram_24
Rank 1 feature: Vertical_Minor_Third_Prevalence
Rank 1 feature: Simple_Initial_Meter
Rank 1 feature: Compound_Initial_Meter
Rank 1 feature: Prevalence_of_Short_Rhythmic_Values
Rank 15 feature: Difference_Between_Most_Common_Rhythmic_Values
Rank 1 feature: Median_Rhythmic_Value_Offset
Rank 1 feature: Beat_Histogram_Tempo_Standardized_3
Rank 1 feature: Beat_Histogram_Tempo_Standardized_4
Rank 1 feature: Beat_Histogram_Tempo_Standardized_12
Rank 1 feature: Beat_Histogram_Tempo_Standardized_21
Rank 1 feature: Beat_Histogram_Tempo_Standardized_56
Rank 1 feature: Mean_Tempo
Rank 1 feature: Note_Density
Rank 1 feature: Average_Time_Between_Attacks
Rank 1 feature: Amount_of_Staccato
Rank 1 feature: Beat_Histogram_15
Rank 1 feature: Number_of_Relatively_Strong_Rhythmic_Pulses
Rank 1 fe

In [14]:
%%time
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=20)
# clf = svm.SVC(C=0.1, kernel='linear', verbose = True)
# #clf = svm.SVC(C=0.1, kernel='rbf', verbose = True)
# #cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=20, random_state = 1)
# #cv = StratifiedKFold(5)
# clf = clf.fit(rfe_X_train, Y_train3.values.ravel())
# train_accuracy_svm_rfe = cross_val_score(estimator=clf,X=rfe_X_train,y=Y_train3.values.ravel(), cv=cv, n_jobs=15).mean()
# test_accuracy_svm_rfe = clf.score(rfe_X_test, Y_test3.values.ravel())

param = {'C': [0.001,0.01,0.1, 1, 10, 100, 1000],
              'kernel': ['linear']}
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=20)
grid = GridSearchCV(svm.SVC(), param, verbose = 3, cv=cv, return_train_score=True, n_jobs=15)
grid.fit(rfe_X_train, Y_train3.values.ravel())
    
train_accuracy_svm_rfe = grid.best_score_
    
grid_predict3 = grid.predict(rfe_X_test)
test_accuracy_svm_rfe  = metrics.accuracy_score(Y_test3,grid_predict3)

print("Training accuracy " + str(train_accuracy_svm_rfe))
print("Test accuracy " + str(test_accuracy_svm_rfe))
print(grid.best_params_)

Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Training accuracy 0.48467032967032964
Test accuracy 0.3898305084745763
{'C': 1, 'kernel': 'linear'}
Wall time: 965 ms


https://machinelearningmastery.com/feature-selection-with-real-and-categorical-data/ **ANOVA F measure**

In [15]:
%%time
#clf = svm.SVC(C=0.1, kernel='linear')
#clf = LinearSVC(C=0.1, max_iter = 10000000)
param = {'C': [0.001,0.01,0.1, 1, 10, 100, 1000],
              'kernel': ['linear']}
num_features = []
train_acc_F_measure = []
test_acc_F_measure = []
best_params = []
xcols = np.array(xcols)
for count in range(1,1496):
    #clf = svm.SVC(C=0.1, kernel='linear')
    #clf = svm.SVC(C=0.1, kernel='rbf')
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=20)
    test = SelectKBest(score_func=f_classif, k = count)
    X_F_measure = test.fit_transform(normalized_X,Y.values.ravel())
    #X_F_measure = test.fit_transform(X_train3,Y_train3.values.ravel())
    #X_test_F_measure = test.fit_transform(X_test3,Y_test3.values.ravel())
    f_xcols = xcols[test.get_support()]
    X_F_measure = pd.DataFrame(X_F_measure, columns=f_xcols)
    #X_test_F_measure = pd.DataFrame(X_test_F_measure, columns=f_xcols)
    X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X_F_measure, Y, test_size=0.3,stratify = Y)
#     clf.fit(X_train2, Y_train2.values.ravel())
#     train_accuracy_svm_f = cross_val_score(clf, X_train2, Y_train2.values.ravel(), cv=cv, n_jobs=15).mean()
#     test_accuracy_svm_f = clf.score(X_test2, Y_test2.values.ravel())

    grid = GridSearchCV(svm.SVC(), param, verbose = 3, cv=cv, return_train_score=True, n_jobs=15)
    grid.fit(X_train2, Y_train2.values.ravel())
    
    train_accuracy_svm_f = grid.best_score_
    
    grid_predict = grid.predict(X_test2)
    test_accuracy_svm_f = metrics.accuracy_score(Y_test2,grid_predict)
    
    num_features.append(count)
    train_acc_F_measure.append(train_accuracy_svm_f)
    test_acc_F_measure.append(test_accuracy_svm_f)
    best_params.append(grid.best_params_)

Fitting 200 folds for each of 7 candidates, totalling 1400 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [   1    2    3    4    5    6    7    8    9   10   15   18   20  113
  114  116  117  118  119  120  121  122  123  124  125  127  259  266
  270  271  272  273  274  276  277  278  279  280  281  282  283  284
  285  286  287  288  289  290  291  292  293  294  295  296  297  298
  299  300  301  302  303  304  305  306  307  308  309  310  311  312
  313  314  315  316  317  423  425  427  428  430  431  432  433  434
  435  436  437  438  439  440  441  442  443  444  445  446  447  448
  449  450  451  452  453  454  455  456  457  458  459  460  461  462
  463  464  465  466  467  468  469  529  982 1018 1019 1044 1046 1057
 1063 1065 1071 1075 1076 1078 1081 1082 1083 1086 1088 1095 1098 1138
 1157 1193 1194 1219 1221 1232 1238 1240 1246 1250 1251 1253 1256 1257
 1258 1261 1263 1270 1273 1313 1332 1368 1369 1394 1396 1407 1413 1415
 1421 1422 1425 1426 1428 1

Fitting 200 folds for each of 7 candidates, totalling 1400 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [   1    2    3    4    5    6    7    8    9   10   15   18   20  113
  114  116  117  118  119  120  121  122  123  124  125  127  259  266
  270  271  272  273  274  276  277  278  279  280  281  282  283  284
  285  286  287  288  289  290  291  292  293  294  295  296  297  298
  299  300  301  302  303  304  305  306  307  308  309  310  311  312
  313  314  315  316  317  423  425  427  428  430  431  432  433  434
  435  436  437  438  439  440  441  442  443  444  445  446  447  448
  449  450  451  452  453  454  455  456  457  458  459  460  461  462
  463  464  465  466  467  468  469  529  982 1018 1019 1044 1046 1057
 1063 1065 1071 1075 1076 1078 1081 1082 1083 1086 1088 1095 1098 1138
 1157 1193 1194 1219 1221 1232 1238 1240 1246 1250 1251 1253 1256 1257
 1258 1261 1263 1270 1273 1313 1332 1368 1369 1394 1396 1407 1413 1415
 1421 1422 1425 1426 1428 1

Fitting 200 folds for each of 7 candidates, totalling 1400 fits


...

Fitting 200 folds for each of 7 candidates, totalling 1400 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [   1    2    3    4    5    6    7    8    9   10   15   18   20  113
  114  116  117  118  119  120  121  122  123  124  125  127  259  266
  270  271  272  273  274  276  277  278  279  280  281  282  283  284
  285  286  287  288  289  290  291  292  293  294  295  296  297  298
  299  300  301  302  303  304  305  306  307  308  309  310  311  312
  313  314  315  316  317  423  425  427  428  430  431  432  433  434
  435  436  437  438  439  440  441  442  443  444  445  446  447  448
  449  450  451  452  453  454  455  456  457  458  459  460  461  462
  463  464  465  466  467  468  469  529  982 1018 1019 1044 1046 1057
 1063 1065 1071 1075 1076 1078 1081 1082 1083 1086 1088 1095 1098 1138
 1157 1193 1194 1219 1221 1232 1238 1240 1246 1250 1251 1253 1256 1257
 1258 1261 1263 1270 1273 1313 1332 1368 1369 1394 1396 1407 1413 1415
 1421 1422 1425 1426 1428 1

Fitting 200 folds for each of 7 candidates, totalling 1400 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [   1    2    3    4    5    6    7    8    9   10   15   18   20  113
  114  116  117  118  119  120  121  122  123  124  125  127  259  266
  270  271  272  273  274  276  277  278  279  280  281  282  283  284
  285  286  287  288  289  290  291  292  293  294  295  296  297  298
  299  300  301  302  303  304  305  306  307  308  309  310  311  312
  313  314  315  316  317  423  425  427  428  430  431  432  433  434
  435  436  437  438  439  440  441  442  443  444  445  446  447  448
  449  450  451  452  453  454  455  456  457  458  459  460  461  462
  463  464  465  466  467  468  469  529  982 1018 1019 1044 1046 1057
 1063 1065 1071 1075 1076 1078 1081 1082 1083 1086 1088 1095 1098 1138
 1157 1193 1194 1219 1221 1232 1238 1240 1246 1250 1251 1253 1256 1257
 1258 1261 1263 1270 1273 1313 1332 1368 1369 1394 1396 1407 1413 1415
 1421 1422 1425 1426 1428 1

Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Wall time: 2h 5min 18s


In [17]:
# best_tacc_f = max(test_acc_F_measure)
# nfeats = num_features[test_acc_F_measure.index(best_tacc_f)]
# corr_train_f = train_acc_F_measure[test_acc_F_measure.index(best_tacc_f)]

best_train_f = max(train_acc_F_measure)
nfeats = num_features[train_acc_F_measure.index(best_train_f)]
best_tacc_f = test_acc_F_measure[train_acc_F_measure.index(best_train_f)]
param = best_params[train_acc_F_measure.index(best_train_f)]

print("Number of features: " + str(nfeats))
print("Train Accuracy: " + str(best_train_f))
print("Test Accuracy: " + str(best_tacc_f))
print("Param: " + str(param))

Number of features: 218
Train Accuracy: 0.5756043956043956
Test Accuracy: 0.3728813559322034
Param: {'C': 0.1, 'kernel': 'linear'}


#  Chi squared test

In [ ]:
%%time
param = {'C': [0.001,0.01,0.1, 1, 10, 100, 1000],
              'kernel': ['linear']}
#clf = svm.SVC(C=0.1, kernel='linear')
#clf = LinearSVC(C=0.1, max_iter = 10000000)
num_features = []
train_acc_c_measure = []
test_acc_c_measure = []
best_params = []
xcols = np.array(xcols)
for count in range(1,1496):
    #clf = svm.SVC(C=0.1, kernel='linear')
    #clf = svm.SVC(C=0.1, kernel='rbf')
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=20)
    test = SelectKBest(score_func=chi2, k = count)
    X_c_measure = test.fit_transform(normalized_X,Y.values.ravel())
    #X_c_measure = test.fit_transform(X_train3,Y_train3.values.ravel())
    #X_test_c_measure = test.fit_transform(X_test3,Y_test3.values.ravel())
    c_xcols = xcols[test.get_support()]
    X_c_measure = pd.DataFrame(X_c_measure, columns=c_xcols)
    #X_test_c_measure = pd.DataFrame(X_test_c_measure, columns=c_xcols)
    X_train2, X_test2, Y_train2, Y_test2 = train_test_split(X_c_measure, Y, test_size=0.3,stratify = Y)
#     clf.fit(X_train2, Y_train2.values.ravel())
#     train_accuracy_svm_c = cross_val_score(clf, X_train2, Y_train2.values.ravel(), cv=cv, n_jobs=15).mean()
#     test_accuracy_svm_c = clf.score(X_test2, Y_test2.values.ravel())

    grid = GridSearchCV(svm.SVC(), param, verbose = 3, cv=cv, return_train_score=True, n_jobs=15)
    grid.fit(X_train2, Y_train2.values.ravel())
    
    train_accuracy_svm_c = grid.best_score_
    
    grid_predict = grid.predict(X_test2)
    test_accuracy_svm_c = metrics.accuracy_score(Y_test2,grid_predict)
    
    num_features.append(count)
    train_acc_c_measure.append(train_accuracy_svm_c)
    test_acc_c_measure.append(test_accuracy_svm_c)
    best_params.append(grid.best_params_)

Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidat

Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidat

Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidat

Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidat

Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidat

...

Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidat

Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidat

Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidat

Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidat

Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidates, totalling 1400 fits
Fitting 200 folds for each of 7 candidat

In [ ]:
best_train_c = max(train_acc_c_measure)
nfeats = num_features[train_acc_c_measure.index(best_train_c)]
best_tacc_c = test_acc_c_measure[train_acc_c_measure.index(best_train_c)]
param = best_params[train_acc_c_measure.index(best_train_c)]

print("Number of features: " + str(nfeats))
print("Train Accuracy: " + str(best_train_c))
print("Test Accuracy: " + str(best_tacc_c))
print("Param: " + str(param))

#  Graph Maker

In [ ]:
train_accuracy_list = [train_accuracy_svm_norm, train_accuracy_svm_rf,train_accuracy_svm_rfe, best_train_f, best_train_c ]
test_accuracy_list = [test_accuracy_svm_norm, test_accuracy_svm_rf,test_accuracy_svm_rfe, best_tacc_f, best_tacc_c ]
labels = ['Base', 'Tree Based', 'RFE', 'F test','Chi-squared test']

print(labels)
print(train_accuracy_list)
print(test_accuracy_list)

fig1,ax1 = plt.subplots()
ax1.set_title("Training Accuracy for SVM Classifier with feature selection")
ax1.bar(labels, train_accuracy_list)
plt.savefig("Train_Accuracy_linear_grid.png")

fig2,ax2 = plt.subplots()
ax2.set_title("Testing Accuracy for SVM Classifier with feature selection")
ax2.bar(labels, test_accuracy_list, color='r')
plt.savefig("Test_Accuracy_linear_grid.png")